In [16]:
import pandas as pd
import math

df = pd.read_csv("hos.csv")

In [17]:
df.loc[df.phone_number.isin(['无','暂无','未知','0','<p>']),"phone_number"] = pd.NA
df.loc[df.position.isin(['无','暂无','未知','0','<p>']),"phone_number"] = pd.NA
df.loc[df.intro.isin(['无','无。','暂无介绍','暂无介绍。','暂无','暂无。','<p>','未知','0']),"intro"] = pd.NA
df.intro = df.intro.fillna("")

In [18]:
# 详细的地理信息
df.insert(loc=2, column='city', value=pd.NA)
df.insert(loc=3, column='area', value=pd.NA)
df.insert(loc=7, column='telephone_from_baidu', value=pd.NA)
df.insert(loc=9, column='address_from_baidu', value=pd.NA)
df.columns

Index(['_id', 'departments', 'city', 'area', 'evaluation', 'followers',
       'img_url', 'telephone_from_baidu', 'intro', 'address_from_baidu',
       'name', 'phone_number', 'position', 'reservation', 'special', 'url'],
      dtype='object')

In [25]:
import requests
import json

ak = "HGHvGvZ2Cz27mR0Niv1umxRngAjDLkBg"
output = "json"
fail_list = []
for i in range(0,df.shape[0]):
    if i % 100 == 0:
        print(i)
    name = df.iloc[i].loc["name"]
    print(name)
    # region = df.iloc[i].loc["province"]
    url = "https://api.map.baidu.com/place/v2/search?query={}&region={}&output={}&ak={}".format(name, '全国', output, ak)
    response = requests.get(url)
    tx = json.loads(response.text)
    if "results" in tx.keys() and len(tx["results"]) > 0:
        print(tx["results"])
        if "city" in tx["results"][0].keys():
            df.loc[i,"city"] = tx["results"][0]["city"]
        if "area" in tx["results"][0].keys():
            df.loc[i,"area"] = tx["results"][0]["area"]
        if "telephone" in tx["results"][0].keys():
            df.loc[i,"telephone_from_baidu"] = tx["results"][0]["telephone"]
        if "address" in tx["results"][0].keys():
            df.loc[i,"address_from_baidu"] = tx["results"][0]["address"]
    else:
        fail_list.append(name)
        print("fail:{}".format(name))


0
北京协和医院
[{'name': '北京协和医院(东单院区)', 'location': {'lat': 39.918962, 'lng': 116.422135}, 'address': '北京市东城区帅府园1号', 'province': '北京市', 'city': '北京市', 'area': '东城区', 'street_id': '210827b26e32b09b167643ca', 'telephone': '(010)69156114', 'detail': 1, 'uid': '210827b26e32b09b167643ca'}, {'name': '北京协和医院(东单院区)-南门', 'location': {'lat': 39.916947, 'lng': 116.422265}, 'address': '北京市东城区帅府园1号', 'province': '北京市', 'city': '北京市', 'area': '东城区', 'detail': 1, 'uid': '7b133973ef88dddf2e47337e'}, {'name': '北京协和医院(东单院区)-东1门', 'location': {'lat': 39.919699, 'lng': 116.424119}, 'address': '北京市东城区帅府园1号', 'province': '北京市', 'city': '北京市', 'area': '东城区', 'detail': 1, 'uid': 'ef85b0040b36a0d8c9c3db5a'}, {'name': '北京协和医院(东单院区)-东北门', 'location': {'lat': 39.920338, 'lng': 116.424165}, 'address': '北京市东城区帅府园1号', 'province': '北京市', 'city': '北京市', 'area': '东城区', 'detail': 1, 'uid': '97086417ab4793a32fb96d22'}, {'name': '北京协和医院(西单院区)', 'location': {'lat': 39.918891, 'lng': 116.374086}, 'address': '北京市西城区大木仓胡同41号', 'pr

In [26]:
# df.insert(loc=11, column='n_reservation', value=0)
# df.insert(loc=13, column='n_evaluation', value=0)
# df.insert(loc=15, column='n_followers', value=0)
df["n_reservation"] = df.reservation.apply(lambda x: int(eval(x)) if "万" not in x else int(eval(x[:x.find("万")]))*10000)
df["n_evaluation"] = df.evaluation.apply(lambda x: int(eval(x)) if "万" not in x else int(eval(x[:x.find("万")]))*10000)
df["n_followers"] = df.followers.apply(lambda x: int(eval(x)) if "万" not in x else int(eval(x[:x.find("万")]))*10000)

In [27]:
# intro
print(df.intro.isna().sum()) # null
print(df[df.intro.isin(['无','无。','暂无介绍','暂无介绍。','暂无','暂无。','<p>','未知','0'])].shape[0])

0
0


In [28]:
df.loc[df.intro.isin(['无','无。','暂无介绍','暂无介绍。','暂无','暂无。','<p>','未知','0']),"intro"] = pd.NA
df.intro = df.intro.fillna("")

In [29]:
df.intro = df.intro.apply(lambda x: x[:x.rfind("。")+1] if x.find("。")!=-1 else x+"。")
df.intro = df.intro.apply(lambda x: x[1:] if x.startswith("\"") else x)
df.intro = df.intro.apply(lambda x: x.replace(" ","") if x.startswith(" ") else x)
for idx, row in df.iterrows():
    if row["intro"] == "。":
        df.loc[idx,"intro"] = ""

In [ ]:
# Reordering the columns as per the suggested logical order
column_order = [
    '_id', 'name', 'city', 'area', 'address_from_baidu', 'phone_number', 'telephone_from_baidu',
    'departments', 'special', 'evaluation', 'followers', 'reservation', 'intro', 'img_url',
    'position', 'url', 'n_reservation', 'n_evaluation', 'n_followers'
]

# Rearranging the columns in the dataset
ordered_data = df[column_order]
ordered_data.to_csv("Hospital_Data.csv", index=False)
